In [1]:
from functools import partial
import numpy as np
from scipy.optimize import root_scalar, minimize, newton, root_scalar, brentq, basinhopping
import nidaqmx as ni
from K10CR1 import K10CR1_stage
import matplotlib.pyplot as plt
import pyvisa as pv
from tqdm import tqdm
import time
import csv

def read(DAQ_name='Dev2', verbose=False, nsamples=1000):

    with ni.Task() as read_task:
        
        read_task.ai_channels.add_ai_voltage_chan(DAQ_name + "/ai16",
                                    min_val=-10, max_val=10)
        result = np.mean(np.array(read_task.read(nsamples)))
    if verbose:
        print(result)
    return result

In [2]:
rm = pv.ResourceManager()
Laser = rm.open_resource('USB0::0x1313::0x804F::M00332686::INSTR')
Laser.write(f"source1:current:level:amplitude 0.07")


hwp = K10CR1_stage(serial_no=55422054, verbose=False)
hwp.initialize_instrument()
hwp.home()
qwp = K10CR1_stage(serial_no=55425654, verbose=False)
qwp.initialize_instrument()
qwp.home()
pol = K10CR1_stage(serial_no=55425494, verbose=False)
pol.initialize_instrument()
pol.home()

Important: make sure you are not running Kinesis software in the meantime. 
Otherwise the initialization will fail.
Important: make sure you are not running Kinesis software in the meantime. 
Otherwise the initialization will fail.
Important: make sure you are not running Kinesis software in the meantime. 
Otherwise the initialization will fail.


In [3]:
def rotate(optic,
           Laser=Laser,
           start=0,
           end=180, 
           num=31,
           plot=True):
    
    Laser.write('output:state 1')
    angles = np.linspace(start,end,num)
    readings = np.zeros(shape=(num))
    
    # Stablize
    time.sleep(1)

    for index, angle in enumerate(angles):
        optic.move(angle)
        reading = read()
        readings[index] = reading
    
    Laser.write('output:state 0')

    if plot: 
        fig, ax = plt.subplots(subplot_kw={'projection':'polar'})
        ax.plot(angles / 180.0 * np.pi, readings)
        ax.set_ylim(0,)
        ax.set_title('hwp' if optic == hwp else 'qwp')
        plt.show()

    return angles * np.pi / 180, readings

In [4]:
def optimize(pol_angle, tolerance=0.01, max_iter=6, plot=True):
    
    pol.move(pol_angle)

    print(f'Pol angle = {pol_angle}')
    
    min_readings = []

    index = 0
    
   

    angles, readings = rotate(optic=hwp, plot=plot)
    min_reading = np.min(readings)
    min_hwp_angle = angles[np.argmin(readings)] * 180.0 / np.pi
    min_readings.append(min_reading)
    print(min_reading)

    hwp.move(min_hwp_angle)
    angles, readings = rotate(optic=qwp, plot=plot)
    min_reading = np.min(readings)
    min_qwp_angle = angles[np.argmin(readings)] * 180.0 / np.pi
    min_readings.append(min_reading)
    print(min_reading)

    relative_change = 1.0
    

        

    
    while min_reading >= tolerance and index <= max_iter and relative_change > 0.01:

        qwp.move(min_qwp_angle)
        scan_range = 20.0/(2 + index)
        angles, readings = rotate(optic=hwp, start=min_hwp_angle-scan_range, end=min_hwp_angle+scan_range, num=21, plot=False)
        min_reading = np.min(readings)
        min_hwp_angle = angles[np.argmin(readings)] * 180.0 / np.pi
        min_readings.append(min_reading)
        print(min_reading)

        hwp.move(min_hwp_angle)
        angles, readings = rotate(optic=qwp, start=min_qwp_angle-scan_range, end=min_qwp_angle+scan_range, num=21, plot=False)
        current_min_reading = np.min(readings)
        relative_change = np.abs((current_min_reading - min_reading)/min_reading)
        min_reading = current_min_reading
        min_qwp_angle = angles[np.argmin(readings)] * 180.0 / np.pi
        min_readings.append(min_reading)
        print(min_reading)

        index += 1

    pol.home()
    hwp.home()
    qwp.home()
    
    return (pol_angle, min_qwp_angle, min_hwp_angle), min_readings

In [ ]:
pol_angles = np.linspace(0, 180, num=12, endpoint=False)
results = []
all_readings = []


for index, pol_angle in enumerate(pol_angles):
    print(f'Index = {index}')
    angles, readings = optimize(pol_angle=pol_angle, plot=True)
    results.append(angles)
    all_readings.append(readings)
    guess = (angles[1], angles[2])




In [7]:
np.save('angles_pol_qwp_hwp3', np.array(results))
with open('all_readings3', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    for row in all_readings:
        write.writerow(row)